Simple neural net to represent sphere on plane ambient occlusion texture
<br/>Inputs: u, v
<br/>Output: occlusion factor

In [2]:
import keras
import imageio.v2 as imageio
import numpy as np
import random
from PIL import Image

In [3]:
# get data 
# img = imageio.imread('../resources/textures/sphereTex.png')
# img = img[:, :, 0] # convert to grayscale
# rows = len(img) # rows
# cols = len(img[0]) # cols

# read data
# data = []
# with open("out.txt", "r") as text_file:
#     line = text_file.read()
#     data = line.split('\n')[:-1]
#     data = [ list(map(float, x.split(','))) for x in data ]
# data = np.array(data)

# # split data
# np.random.shuffle(data)
# partition = int(0.8 * len(data))
# X_train = data[:partition, :-1]
# y_train = data[:partition, -1] / 255.0
# X_test = data[partition:, :-1] 
# y_test = data[partition:, -1] / 255.0

In [ ]:
# read data
data = []
with open("../resources/occlusion/data.txt", "r") as text_file:
    line = text_file.read()
    data = line.split('\n')[:-1]
    data = [ list(map(float, x.split(','))) for x in data ]
data = np.array(data)

# split data
np.random.shuffle(data)
partition = int(0.8 * len(data))
X_train = data[:partition, :-1]
y_train = data[:partition, -1]
X_test = data[partition:, :-1] 
y_test = data[partition:, -1]

: 

In [4]:
# build model
model = keras.Sequential([
    keras.layers.Dense(512, input_shape=(2,), activation='tanh'), # hidden
    keras.layers.Dense(512, activation='tanh'), # hidden
    keras.layers.Dense(1) # output
])
model.compile(optimizer="rmsprop", loss=keras.losses.MeanSquaredError(), metrics="mse")

In [5]:
# train model
history = model.fit(X_train, y_train, epochs=40, batch_size=64)

Epoch 1/40
300/300 [==============================] - 1s 1ms/step - loss: 0.0645 - mse: 0.0645
Epoch 2/40
300/300 [==============================] - 0s 2ms/step - loss: 0.0278 - mse: 0.0278
Epoch 3/40
300/300 [==============================] - 0s 1ms/step - loss: 0.0260 - mse: 0.0260
Epoch 4/40
300/300 [==============================] - 0s 1ms/step - loss: 0.0251 - mse: 0.0251
Epoch 5/40
300/300 [==============================] - 0s 2ms/step - loss: 0.0118 - mse: 0.0118
Epoch 6/40
300/300 [==============================] - 0s 1ms/step - loss: 0.0037 - mse: 0.0037
Epoch 7/40
300/300 [==============================] - 0s 1ms/step - loss: 0.0031 - mse: 0.0031
Epoch 8/40
300/300 [==============================] - 0s 1ms/step - loss: 0.0029 - mse: 0.0029
Epoch 9/40
300/300 [==============================] - 0s 1ms/step - loss: 0.0027 - mse: 0.0027
Epoch 10/40
300/300 [==============================] - 0s 1ms/step - loss: 0.0026 - mse: 0.0026
Epoch 11/40
300/300 [============================

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               1536      
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 1)                 513       
                                                                 
Total params: 264705 (1.01 MB)
Trainable params: 264705 (1.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
# generate texture using model
dim = 512
inputs = []
for r in range(dim,0,-1):
    for c in range(dim):
        x = [c/(dim-1), r/(dim-1)]
        inputs.append(x)
outputs = model.predict(inputs)
outputs = [ 255 if y[0] > 1 else 255*y[0] for y in outputs ]

8192/8192 [==============================] - 3s 375us/step


In [8]:
# save image
img = Image.new('L', (dim,dim))
img.putdata(outputs)
img.save('nnTexture.png')

In [9]:
# manual evaluation
wab = model.get_weights()
w1 = wab[0]
b1 = wab[1]
w2 = wab[2]
b2 = wab[3]
w3 = wab[4]
b3 = wab[5]

z1 = keras.activations.tanh(np.matmul(np.array([0.5, 0.5]), w1) + b1)
z2 = keras.activations.tanh(np.matmul(z1, w2) + b2)
z3 = keras.activations.tanh(np.matmul(z2, w3) + b3)
res = z3.numpy()[0]

In [24]:
res

0.755876855928253

In [22]:
# save weights and biases as buffer
output_buf = str(len(model.layers)) + "\n" # layer count

for i in range(0, len(wab)-1, 2): # iterate through layers
    weights = wab[i]
    output_buf += str(weights.shape[0]) + " " + str(weights.shape[1]) + "\n" # input size output size
    for neuron in weights: # iterate through
        for edges in neuron:
            output_buf += str(edges) + " "
    for biases in wab[i+1]:
        output_buf += str(biases) + " "
    output_buf = output_buf[:-1] + "\n" # ignore extraneous space and end line
    

In [23]:
with open("../resources/evaluators/model.txt", "w") as text_file:
    text_file.write(output_buf) # add more digits